##### Imports and Variables

In [ ]:
from collate import drop_cast_and_create_taxi, weather_process
from _global_vars import *

Taxi data preprocessing

In [ ]:
# Read in 2019 taxi data & inspect
sdf_full = spark.read.parquet('../data/raw/tlc_data/tlc_data_yellow/2019*')
#sdf_full.show(1, vertical=True)

In [ ]:
# inspect-pre
print(sdf_full.count(), len(sdf_full.columns))
sdf_full.printSchema()
#sdf_full.limit(3)

In [6]:
# Call helper function to select and cast useful features to appropriate dtypes
sdf = drop_cast_and_create_taxi(sdf_full)

In [ ]:
# inspect-post
print(sdf.count(), len(sdf.columns))
sdf.printSchema()

In [ ]:
# Pre-cleaning descriptive statistics
sdf.describe(*non_categorical_features).show()

Weather aggregation, preprocessing and exporting

In [9]:
# Read in weather csv
weather_date_path = "../data/raw/nyc_weather_date/NYC.csv"
weather_sdf = spark.read.csv(weather_date_path, header=True, inferSchema=True)

In [10]:
# inspect-pre
print(weather_sdf.count(), len(weather_sdf.columns))
weather_sdf.printSchema()

root
 |-- station: string (nullable = true)
 |-- valid: string (nullable = true)
 |-- lon: double (nullable = true)
 |-- lat: double (nullable = true)
 |-- tmpf: string (nullable = true)
 |-- dwpf: string (nullable = true)
 |-- relh: string (nullable = true)
 |-- drct: string (nullable = true)
 |-- sknt: string (nullable = true)
 |-- p01i: string (nullable = true)
 |-- alti: string (nullable = true)
 |-- mslp: string (nullable = true)
 |-- vsby: string (nullable = true)
 |-- gust: string (nullable = true)
 |-- skyc1: string (nullable = true)
 |-- skyc2: string (nullable = true)
 |-- skyc3: string (nullable = true)
 |-- skyc4: string (nullable = true)
 |-- skyl1: string (nullable = true)
 |-- skyl2: string (nullable = true)
 |-- skyl3: string (nullable = true)
 |-- skyl4: string (nullable = true)
 |-- wxcodes: string (nullable = true)
 |-- ice_accretion_1hr: string (nullable = true)
 |-- ice_accretion_3hr: string (nullable = true)
 |-- ice_accretion_6hr: string (nullable = true)
 |-- pe

In [12]:
# Call helper function to preprocess weather data
weather_sdf = weather_process(weather_sdf)

In [ ]:
# inspect-post
print(weather_sdf.count(), len(weather_sdf.columns))
weather_sdf.printSchema()

In [15]:
# Export cleaned weather data
weather_sdf.write.mode('overwrite').csv(export_relative_dir + "weather_data_clean.csv")

Merge taxi and weather data on $\textit{hour of day}$ feature

In [ ]:
# Merge
combined_sdf = sdf.join(weather_sdf,
                        on=['hour_of_day_of_year'],
                        how='inner')

In [ ]:
combined_sdf.write.mode('overwrite').parquet(export_relative_dir + "yt2019_feature_eng.parquet")
print("Export completed...")

In [18]:
spark.sql("CLEAR CACHE")
spark.stop()